<a href="https://colab.research.google.com/github/Tarsha777/Drone_detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install inference-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 760.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.12.15
    Uninstalling aiohttp-3.12.15:
      Successfully uninstalled aiohttp-3.12.15


In [3]:
from inference_sdk import InferenceHTTPClient
import cv2
import numpy as np

client = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="NT4m2Iai7kZ16rXEgDPI"
)


In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os

image_folder_path = "/content/drive/MyDrive/flood_dataset"

all_entries = os.listdir(image_folder_path)

image_files = [entry for entry in all_entries if os.path.isfile(os.path.join(image_folder_path, entry)) and entry.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(image_files)} image files.")

Found 90 image files.


In [7]:
print(os.listdir('/content/drive/MyDrive'))

['Colab Notebooks', 'agent_performance_summary.csv', 'dpdzero.py', 'WhatsApp Image 2025-05-04 at 18.36.31_8de23d06.jpg', 'DPDzero_report.pdf', 'Datasets', 'flood_dataset', 'flood_dataset_labeling_results.json', 'object_detection_report.md']


In [8]:
image_folder_path = "/content/drive/MyDrive/flood_dataset"

all_entries = os.listdir(image_folder_path)

image_files = [entry for entry in all_entries if os.path.isfile(os.path.join(image_folder_path, entry)) and entry.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(image_files)} image files.")

Found 90 image files.


## Iterate and label

### Subtask:
Loop through each image file, load it, and run the labeling workflow using the `InferenceHTTPClient`.


In [9]:
import cv2
import numpy as np
import os

labeling_results = {}

for image_filename in image_files:
    image_path = os.path.join(image_folder_path, image_filename)
    try:
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image {image_filename}")
            continue

        # Convert the image to a numpy array (OpenCV loads images as numpy arrays)
        image_array = np.array(image)

        result = client.run_workflow(
            workspace_name="labeling-gymly",
            workflow_id="find-garbage-vehicles-floodeds-and-buildings",
            images={
                "image": image_array
            },
            use_cache=True
        )
        labeling_results[image_filename] = result
        print(f"Successfully processed {image_filename}")
    except Exception as e:
        print(f"Error processing {image_filename}: {e}")


Successfully processed chip_0_0DSC07598.JPG
Successfully processed chip_0_0DSC07604.JPG
Successfully processed chip_0_0DSC07614.JPG
Successfully processed chip_0_0DSC07745.JPG
Successfully processed chip_0_0DSC07702.JPG
Successfully processed chip_0_0DSC07602.JPG
Successfully processed chip_0_0DSC07557.JPG
Successfully processed chip_0_0DSC07683.JPG
Successfully processed chip_0_0DSC07748.JPG
Successfully processed chip_0_0DSC07460.JPG
Successfully processed chip_0_0DSC07697.JPG
Successfully processed chip_0_0DSC07751.JPG
Successfully processed chip_0_0DSC07609.JPG
Successfully processed chip_0_0DSC07698.JPG
Successfully processed chip_0_0DSC07605.JPG
Successfully processed chip_0_0DSC07700.JPG
Successfully processed chip_0_0DSC07699.JPG
Successfully processed chip_0_0DSC07693.JPG
Successfully processed chip_0_0DSC07695.JPG
Successfully processed chip_0_0DSC07507.JPG
Successfully processed chip_0_0DSC07556.JPG
Successfully processed chip_0_0DSC07458.JPG
Successfully processed chip_0_0D

In [ ]:
import os
import shutil
import yaml
import cv2
import numpy as np
import json # Import json to load the results
from sklearn.model_selection import train_test_split # Import train_test_split

# Define the path to the image folder (already defined in previous steps)
image_folder_path = "/content/drive/MyDrive/flood_dataset"
output_file_path = "/content/drive/MyDrive/flood_dataset_labeling_results.json"


# Load the labeling results
try:
    with open(output_file_path, 'r') as f:
        labeling_results = json.load(f)
    print(f"Successfully loaded labeling results for {len(labeling_results)} images from {output_file_path}")
except FileNotFoundError:
    print(f"Error: The file {output_file_path} was not found.")
    labeling_results = None # Set to None if loading fails
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {output_file_path}. The file might be corrupted.")
    labeling_results = None
except Exception as e:
    print(f"An unexpected error occurred while loading the file: {e}")
    labeling_results = None

# Split data into training and testing sets
train_data = {}
test_data = {}

if labeling_results:
    image_filenames_list = list(labeling_results.keys())
    labeling_results_list = [labeling_results[filename] for filename in image_filenames_list]

    X_train, X_test, y_train, y_test = train_test_split(
        image_filenames_list, labeling_results_list, test_size=0.2, random_state=42
    )

    for i in range(len(X_train)):
        train_data[X_train[i]] = y_train[i]

    for i in range(len(X_test)):
        test_data[X_test[i]] = y_test[i]

    print(f"Recreated train_data with {len(train_data)} items.")
    print(f"Recreated test_data with {len(test_data)} items.")
else:
    print("Labeling results not loaded, cannot recreate train_data and test_data.")


# Define temporary directories for training and validation data
temp_dir = "/content/yolov8_data"
train_img_dir = os.path.join(temp_dir, "train/images")
train_label_dir = os.path.join(temp_dir, "train/labels")
val_img_dir = os.path.join(temp_dir, "valid/images")
val_label_dir = os.path.join(temp_dir, "valid/labels")

# Create the temporary directories
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# Function to save image and YOLO format labels
def save_yolo_format(image, annotations, image_filename, img_dir, label_dir):
    # Save the image
    img_path = os.path.join(img_dir, image_filename)
    cv2.imwrite(img_path, image)

    # Save the annotations in YOLO format
    label_filename = os.path.splitext(image_filename)[0] + '.txt'
    label_path = os.path.join(label_dir, label_filename)

    img_height, img_width, _ = image.shape

    with open(label_path, 'w') as f:
        for annotation in annotations:
            # Convert xyxy to YOLO format (center_x center_y width height normalized)
            x_min, y_min, x_max, y_max = annotation['box']
            class_id = annotation['class_id']

            x_center = (x_min + x_max) / 2.0 / img_width
            y_center = (y_min + y_max) / 2.0 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


# Save training data
print("Saving training data to temporary directories...")
# Ensure train_data is not empty before iterating
if train_data:
    for image_filename, result in train_data.items():
        image_path = os.path.join(image_folder_path, image_filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image {image_filename} for saving training data.")
            continue

        # Extract annotations from the results structure for saving
        annotations = []
        if result and isinstance(result, list) and len(result) > 0 and 'predictions' in result[0] and isinstance(result[0]['predictions'], dict) and 'predictions' in result[0]['predictions'] and isinstance(result[0]['predictions']['predictions'], list):
            predictions_list = result[0]['predictions']['predictions']
            for p in predictions_list:
                if all(key in p for key in ['x', 'y', 'width', 'height', 'confidence', 'class_id', 'class']):
                    x_center = p['x']
                    y_center = p['y']
                    width = p['width']
                    height = p['height']

                    x_min = x_center - width / 2
                    y_min = y_center - height / 2
                    x_max = x_center + width / 2
                    y_max = y_center + height / 2

                    annotations.append({
                        'box': [x_min, y_min, x_max, y_max],
                        'class_id': p['class_id'],
                        'class': p['class'],
                        'confidence': p['confidence']
                    })
        save_yolo_format(image, annotations, image_filename, train_img_dir, train_label_dir)
    print("Training data saved.")
else:
    print("train_data is empty or not available, skipping saving training data.")


# Save validation data (using test data as validation for this task)
print("Saving validation data to temporary directories...")
# Ensure test_data is not empty before iterating
if test_data:
    for image_filename, result in test_data.items():
        image_path = os.path.join(image_folder_path, image_filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image {image_filename} for saving validation data.")
            continue

        # Extract annotations from the results structure for saving
        annotations = []
        if result and isinstance(result, list) and len(result) > 0 and 'predictions' in result[0] and isinstance(result[0]['predictions'], dict) and 'predictions' in result[0]['predictions'] and isinstance(result[0]['predictions']['predictions'], list):
             predictions_list = result[0]['predictions']['predictions']
             for p in predictions_list:
                if all(key in p for key in ['x', 'y', 'width', 'height', 'confidence', 'class_id', 'class']):
                    x_center = p['x']
                    y_center = p['y']
                    width = p['width']
                    height = p['height']

                    x_min = x_center - width / 2
                    y_min = y_center - height / 2
                    x_max = x_center + width / 2
                    y_max = y_center + height / 2

                    annotations.append({
                        'box': [x_min, y_min, x_max, y_max],
                        'class_id': p['class_id'],
                        'class': p['class'],
                        'confidence': p['confidence']
                    })
        save_yolo_format(image, annotations, image_filename, val_img_dir, val_label_dir)
    print("Validation data saved.")
else:
    print("test_data is empty or not available, skipping saving validation data.")


# Update data.yaml with the correct temporary paths
# Assuming data_yaml and yaml_file_path are defined from a previous cell or need to be recreated
# Let's recreate data_yaml and yaml_file_path for robustness
# Get unique class names and map them to class IDs
all_class_ids = set()
all_class_names = set()

# Iterate through both train_data and test_data to get all class names and IDs
if train_data:
    for result in train_data.values():
         if result and isinstance(result, list) and len(result) > 0 and 'predictions' in result[0] and isinstance(result[0]['predictions'], dict) and 'predictions' in result[0]['predictions'] and isinstance(result[0]['predictions']['predictions'], list):
            for p in result[0]['predictions']['predictions']:
                 if all(key in p for key in ['class_id', 'class']):
                    all_class_ids.add(p['class_id'])
                    all_class_names.add(p['class'])

if test_data:
     for result in test_data.values():
         if result and isinstance(result, list) and len(result) > 0 and 'predictions' in result[0] and isinstance(result[0]['predictions'], dict) and 'predictions' in result[0]['predictions'] and isinstance(result[0]['predictions']['predictions'], list):
            for p in result[0]['predictions']['predictions']:
                 if all(key in p for key in ['class_id', 'class']):
                    all_class_ids.add(p['class_id'])
                    all_class_names.add(p['class'])


sorted_class_names = sorted(list(all_class_names))
nc = len(sorted_class_names) # Number of classes

# Create a data.yaml file for YOLOv8
data_yaml = {
    'train': train_img_dir, # Use actual temporary path
    'val': val_img_dir,   # Use actual temporary path
    'nc': nc,
    'names': sorted_class_names
}

# Save the data.yaml file
yaml_file_path = 'data.yaml'
with open(yaml_file_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"Updated data.yaml with train: {data_yaml['train']} and val: {data_yaml['val']}")

# Now attempt to train the model again
from ultralytics import YOLO

# Assuming model_config is defined from a previous cell or needs to be recreated
# Let's recreate a basic model_config for robustness
model_config = {
    'imgsz': 640,  # Image size for training and inference
    'batch': 16,   # Batch size
    'epochs': 50,  # Number of epochs (can be adjusted)
    'cfg': 'yolov8s.yaml', # Using the small YOLOv8 model configuration
    'data': yaml_file_path, # Path to the data.yaml file
    'cache': True # Cache images for faster training
}


# Instantiate a YOLO model using the specified configuration file
model = YOLO(model_config['cfg'])

# Start the model training process
results = model.train(
    data=model_config['data'],
    epochs=model_config['epochs'],
    imgsz=model_config['imgsz'],
    batch=model_config['batch'],
    cache=model_config['cache']
)

print("Model training completed.")

Successfully loaded labeling results for 90 images from /content/drive/MyDrive/flood_dataset_labeling_results.json
Recreated train_data with 72 items.
Recreated test_data with 18 items.
Saving training data to temporary directories...
Training data saved.
Saving validation data to temporary directories...
Validation data saved.
Updated data.yaml with train: /content/yolov8_data/train/images and val: /content/yolov8_data/valid/images
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fract

In [2]:
import os

report_content = """
# Object Detection Model Training Report

## Model Choice

For this object detection task, YOLOv8 (specifically the YOLOv8s model) was chosen. YOLO (You Only Look Once) models are known for their speed and accuracy in real-time object detection, making them suitable for analyzing aerial drone imagery. YOLOv8 is the latest iteration in the YOLO series, offering improved performance and ease of use. Its architecture is well-suited for identifying multiple objects within images, which aligns with the goal of detecting various classes like vehicles, buildings, garbage, and flooded areas in the dataset.

## Dataset Usage and Preprocessing

The dataset used for training and evaluation was obtained from a Google Drive folder located at `/content/drive/MyDrive/flood_dataset`. The initial labeling of this dataset was performed using a Roboflow workflow ("find-garbage-vehicles-floodeds-and-buildings") via the Roboflow Inference HTTP Client. This process provided bounding box annotations and class labels for objects in each image.

After obtaining the labeling results, the dataset was split into training and testing sets with an 80% to 20% ratio using `sklearn.model_selection.train_test_split`. The image filenames and their corresponding labeling results were split to maintain the association between images and their annotations.

To prepare the data for YOLOv8 training, the images and their annotations were saved to temporary directories (`/content/yolov8_data/train/images`, `/content/yolov8_data/train/labels`, `/content/yolov8_data/valid/images`, `/content/yolov8_data/valid/labels`). The annotations, originally in a dictionary format with x, y, width, and height, were converted to the YOLO format (class_id center_x center_y width height, normalized) and saved as text files corresponding to each image. A `data.yaml` file was created and updated to point to these temporary directories and define the class names and counts.

## Training Process

The YOLOv8s model was trained using the prepared dataset. The training was configured with the following parameters:
- **Image Size (`imgsz`):** 640 pixels
- **Batch Size (`batch`):** 16
- **Epochs (`epochs`):** 50
- **Model Configuration (`cfg`):** yolov8s.yaml
- **Data Configuration (`data`):** data.yaml (pointing to the temporary training and validation data)
- **Cache:** Enabled for faster training

The training process involved iterating over the training data for 50 epochs, with the test set being used for validation at the end of each epoch to monitor performance and prevent overfitting.

## Evaluation Results

The trained model was evaluated on the testing dataset. The following evaluation metrics were obtained:
- **Precision (P):** 0.0041
- **Recall (R):** 0.0147
- **mAP@0.5:** 0.0043
- **mAP@0.5:0.95:** 0.0012

These metrics indicate that the model's performance on the testing set is very low. A low Precision suggests that when the model detects an object, it is often incorrect. A low Recall indicates that the model is missing a large percentage of the actual objects present in the images. The low mAP scores, especially mAP@0.5:0.95, confirm that the model is not effectively identifying and localizing objects across different Intersection over Union (IoU) thresholds. This poor performance could be attributed to several factors, including the small size of the dataset, the complexity of the objects and scenes in aerial imagery, or the need for more extensive hyperparameter tuning and potentially a larger model or longer training duration.

## Sample Predictions

Sample predictions were generated on a few images from the testing set using the trained model with a confidence threshold of 0.25. For the selected sample images visualized, the model did not detect any objects at this confidence level. This observation is consistent with the low evaluation metrics, suggesting that the model has low confidence in its predictions on unseen data. Further analysis with a lower confidence threshold might show some detections, but their reliability would likely be low based on the overall evaluation scores. The visualization process successfully displayed the original images alongside the annotated images, although the annotated images appeared identical to the originals due to the absence of detections.

"""

# Define the path to save the report
report_file_path = "/content/drive/MyDrive/object_detection_report.md"

# Save the report to a markdown file
try:
    with open(report_file_path, "w") as f:
        f.write(report_content)
    print(f"Report successfully saved to {report_file_path}")
except Exception as e:
    print(f"Error saving the report: {e}")

Report successfully saved to /content/drive/MyDrive/object_detection_report.md
